In [ ]:
import pandas as pd
from mainnet_launch.constants import *
import requests
import plotly.express as px

import os
from mainnet_launch.data_fetching.tokemak_subgraph import run_query_with_paginate


from mainnet_launch.data_fetching.get_state_by_block import *

# def fetch_end_of_day_WETH_prices():
#     eth_blocks = build_blocks_to_use(ETH_CHAIN)
#     weth_to_usdc_call = Call(
#                 ROOT_PRICE_ORACLE(ETH_CHAIN),
#                 ["getPriceInQuote(address,address)(uint256)", WETH(ETH_CHAIN), USDC(ETH_CHAIN)],
#                 [("WETH_safe_price", safe_normalize_6_with_bool_success)],
#             )

#     daily_weth_to_usdc_price_df = get_raw_state_by_blocks([weth_to_usdc_call], eth_blocks, chain=ETH_CHAIN)
#     return daily_weth_to_usdc_price_df

# # TODO you need a custom method to get the sonic -> USDC price because ETH refers to the price of S on Sonic not ETH


def get_rebalance_volumne_raw_data(autopool: AutopoolConstants):
    query = """
    query getAutopoolRebalances($address: String!, $first: Int!, $skip: Int!) {
      autopoolRebalances(
        first: $first,
        skip: $skip,
        orderBy: timestamp,
        orderDirection: asc,
        where: { autopool: $address }
      ) {
        autopool
        destinationInName
        destinationOutName
        timestamp
        blockNumber
        transactionHash
        tokenOutValueInEth
        tokenOutValueBaseAsset
      }
    }
    """

    if autopool.chain.chain_id == 1:
        api_url = os.environ["TOKEMAK_ETHEREUM_SUBGRAPH_URL"]
    elif autopool.chain.chain_id == 8453:
        api_url = os.environ["TOKEMAK_BASE_SUBGRAPH_URL"]
    elif autopool.chain.chain_id == 146:
        api_url = os.environ["TOKEMAK_SONIC_SUBGRAPH_URL"]

    variables = {"address": autopool.autopool_eth_addr.lower()}
    df = run_query_with_paginate(api_url, query, variables, "autopoolRebalances")
    df["chain_id"] = autopool.chain.chain_id
    df["autopool_name"] = autopool.name
    df['datetime'] = pd.to_datetime(df['timestamp'].astype(int), unit='s', utc=True)
    df['date'] = df['datetime'].dt.date
    return df



def get_sonic_rebalance_volumne_df():


    df = pd.concat(
        [get_rebalance_volumne_raw_data(autopool) for autopool in ALL_AUTOPOOLS if autopool.chain == SONIC_CHAIN], axis=0
    )

    if np.any(df['autopool_name'] != 'sonicUSD'):
        raise ValueError('Need custom logic to get the Sonic `S` price to USDC')
    
    df['block'] = df['blockNumber'].astype(int)
    df['computed_usd_volumne'] = df['tokenOutValueBaseAsset'].apply(lambda x: int(x) / 1e6)

    return df.set_index('datetime').sort_index()


def get_mainnet_rebalance_volumne_df():
    df = pd.concat(
        [get_rebalance_volumne_raw_data(autopool) for autopool in ALL_AUTOPOOLS if autopool.chain == ETH_CHAIN], axis=0
    )
    df['block'] = df['blockNumber'].astype(int)
    df['tokenOutValueInEth_norm'] = df['tokenOutValueInEth'].apply(lambda x: int(x) / 1e18)

    calls = [Call(
            ROOT_PRICE_ORACLE(ETH_CHAIN),
            ["getPriceInQuote(address,address)(uint256)", WETH(ETH_CHAIN), USDC(ETH_CHAIN)],
            [('mainnet_ETH_TO_USDC_price', safe_normalize_6_with_bool_success)],
        )]
    
    blocks = df['blockNumber'].astype(int)

    base_weth_price_df = get_raw_state_by_blocks(calls, blocks, ETH_CHAIN, include_block_number=True)
    full_df = pd.merge(df, base_weth_price_df, on='block')

    full_df['computed_usd_volumne'] = full_df['tokenOutValueInEth_norm'] * full_df['mainnet_ETH_TO_USDC_price']
    return full_df.set_index('datetime').sort_index()



def get_base_rebalance_volume_df():
    df = pd.concat(
        [get_rebalance_volumne_raw_data(autopool) for autopool in ALL_AUTOPOOLS if autopool.chain == BASE_CHAIN], axis=0
    )


    
    df['block'] = df['blockNumber'].astype(int)
    df['tokenOutValueInEth_norm'] = df['tokenOutValueInEth'].apply(lambda x: int(x) / 1e18)

    calls = [Call(
            ROOT_PRICE_ORACLE(BASE_CHAIN),
            ["getPriceInQuote(address,address)(uint256)", WETH(BASE_CHAIN), USDC(BASE_CHAIN)],
            [('base_chain_ETH_to_USDC_price', safe_normalize_6_with_bool_success)],
        )]
    
    blocks = df['blockNumber'].astype(int)

    base_weth_price_df = get_raw_state_by_blocks(calls, blocks, BASE_CHAIN, include_block_number=True)
    full_df = pd.merge(df, base_weth_price_df, on='block')

    full_df['computed_usd_volumne'] = full_df['tokenOutValueInEth_norm'] * full_df['base_chain_ETH_to_USDC_price']
    return full_df.set_index('datetime').sort_index()


base_df = get_base_rebalance_volume_df()
mainnet_df = get_mainnet_rebalance_volumne_df()
sonic_df=  get_sonic_rebalance_volumne_df()

# 3:20

asyncio timeout [0] /Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/multicall/multicall.py line 91 ['_, _, outputs = await self.aggregate.coroutine(args)']
asyncio timeout [0] /Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/multicall/multicall.py line 91 ['_, _, outputs = await self.aggregate.coroutine(args)']
asyncio timeout [0] /Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/multicall/multicall.py line 91 ['_, _, outputs = await self.aggregate.coroutine(args)']
asyncio timeout [0] /Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/multicall/multicall.py line 91 ['_, _, outputs = await self.aggregate.coroutine(args)']
asyncio timeout [0] /Users/pb/Library/Caches/pypoetry/virtualenvs/mainnet-launch-FtycU18g-py3.10/lib/python3.10/site-packages/multicall/multical

In [42]:
df = pd.concat([mainnet_df, sonic_df, base_df], axis=0).reset_index()
df.columns, df.shape



(Index(['datetime', 'autopool', 'destinationInName', 'destinationOutName',
        'timestamp', 'blockNumber', 'transactionHash', 'tokenOutValueInEth',
        'tokenOutValueBaseAsset', 'chain_id', 'autopool_name', 'date', 'block',
        'tokenOutValueInEth_norm', 'mainnet_ETH_TO_USDC_price',
        'computed_usd_volumne', 'calculatored_usd_volumne',
        'base_chain_ETH_to_USDC_price'],
       dtype='object'),
 (2558, 18))

In [47]:
px.bar(df.groupby(['date', 'autopool_name'])['computed_usd_volumne'].sum().reset_index().pivot(columns='autopool_name', index='date', values='computed_usd_volumne'))

In [3]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
df['weth_to_usdc_safe_price'] = df['date'].map(date_to_price)
df['token_value_out_USDC_from_safe_price'] = df.apply(lambda row : (int(row['tokenOutValueInEth']) / 1e18) row['weth_to_usdc_safe_price'] * , axis=1)
df['token_value_out_USDC_from_base_asset'] =  df.apply(lambda row : (int(row['tokenOutValueBaseAsset']) / 1e6), axis=1)

local_df = df.set_index('datetime')


px.scatter(local_df, x='token_value_out_USDC_from_base_asset', y='token_value_out_USDC_from_safe_price')

In [ ]:
local_df[['tokenOutValueBaseAsset', 'token_value_out_USDC_from_base_asset', 'token_value_out_USDC_from_safe_price', 'tokenOutValueInEth']]

,tokenOutValueBaseAsset,token_value_out_USDC_from_base_asset,token_value_out_USDC_from_safe_price,tokenOutValueInEth
datetime,,,,
2025-06-11 16:29:07+00:00,504267000,504.267000,3.514545e+06,1266652408876268785520
2025-06-11 18:51:19+00:00,126066750,126.066750,8.884626e+05,320204500568452475364
2025-06-11 20:29:35+00:00,119989000,119.989000,8.517601e+05,306976844731701769628
2025-06-11 21:23:08+00:00,6942682000,6942.682000,4.928360e+07,17761954666097889615869
2025-06-11 21:49:19+00:00,2580400000,2580.400000,1.872021e+07,6746817752438737347543
...,...,...,...,...
2025-07-01 05:39:42+00:00,21093714000,21093.714000,NaN,68822211910200061632361
2025-07-01 07:39:50+00:00,20000000000,20000.000000,NaN,64822602680492544280000
2025-07-01 09:39:44+00:00,20000000000,20000.000000,NaN,65509464976899350340000
